In [1]:
import torch

In [2]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification

model_name = "monologg/koelectra-v3-small-nsmc"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name)

In [3]:
device = torch.device("cuda")
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

In [4]:
wordsToRemove = ['광고', '협찬']

In [5]:
def filterWords(text) :
    for word in wordsToRemove:
        text = text.replace(word, '')
    return text

In [6]:
def sentimentalAnalysis(text):
    with torch.no_grad():
        encodedInput = tokenizer.encode_plus(filterWords(text), return_tensors="pt").to(device)
        output = model(encodedInput['input_ids'], attention_mask = encodedInput['attention_mask'])
        logits = output.logits
        probabilities = torch.softmax(logits, dim=1)
        #predictedLabel = torch.argmax(probabilities, dim=1)
        return probabilities[0][1].item()

In [16]:
from flask import Flask, request, jsonify
import threading 
import logging
import json
from werkzeug.serving import run_simple
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json(force=True)
        print(data)
        result = sentimentalAnalysis(data)
        return jsonify(result)
    except Exception as e:
        app.logger.error('Error occured at predict(): %s', e)
        return jsonify(-1), 500
   
class FlaskThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.daemon = True
        self.srv = None
        self.ctx = app.test_request_context()
        self.ctx.push()

    def run(self):
        self.srv = run_simple('0.0.0.0', 5000, app, use_debugger=True, use_reloader=False)
        
    def shutdown(self):
        if self.srv is not None:
            self.srv.shutdown()

flaskThread = FlaskThread()
flaskThread.start()

input("Press Enter to stop the Flask app...")
flaskThread.shutdown()

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://163.239.26.24:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -


</a><br>에스피오나지 : <a href="https://bit.ly/EG_Baker_Pants">https://bit.ly/EG_Baker_Pants</a>
<a href="https://www.youtube.com/watch?v=J4v-_jDav9I&amp;t=07m51s">07:51</a> 상의 정보가 궁금해요
형신고있는 구두 제품 알려주세요
<a href="https://www.youtube.com/watch?v=J4v-_jDav9I&amp;t=9m25s">9:25</a> they팬츠에서 살짝 여유롭다는 m사이즈 허리가 조굼 크다는 말씀일까요?
깡형님 라이트 굿핏 하프 밴딩 슬랙스 무슨색입은거에여 ㅠㅠ
뭔 죄다 품절이여;;
형 이제 패션 레벨 콘텐츠 안 하기로 한거야??? <br>대체 왜??


127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /pred

데이는 굿핏히든라인이 ㄹㅇ가성비 좋고 핏 지림
도미넌트 바지에 상의 회색티셔츠 정보알수있을까요?!
1번 반팔셔츠 어디꺼낙요 ?ㅠㅠ
디미트리블랙 바지와 매치한 옥스퍼드 셔츠와 로퍼는 어디 제품인지 여쭤봐도 될까요??ㅎㅎ
깡님 디미트리 바지에 입은 옥스퍼드셔츠 정보좀 알 수 있을까요? 바지랑 같이 셋트로 입을려구요!!
아무리 트렌드라지만... 너무 와이드만 챙기시네요... <br>슬림핏도 좀 챙겨주세요
데이 살려햇더니 다품절이네요 ㅠㅠ 아쉽네요
3번 5번코디 가방 정보좀 알려주세요 ㅠ
도미넌트를 여기서보니반갑네여 품질 디자인 모두굳이라 즐겨입는 브랜드인데
데이 슬랙스 재입고 안될까요 ㅠㅠ
로드존그레이 무조건사게요 님들 개이쁨
깡님 로마 공항에서 봤어요 ㅎㅎㅎㅎ
1번룩 셔츠이름 뭐예요?
쵝오
<a href="https://www.youtube.com/watch?v=J4v-_jDav9I&amp;t=7m35s">7:35</a> 이 가방 어디 제품인지 알 수 있을까요?😢😢😢


127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:48:07] "POST /predict HTTP/1.1" 200 -


...

형.. 이번 여름 비 많이 온다던데 비올때는 어떤 신발이 좋아..?
바로 노메뉴얼 가방 질럿습니다! 감사합니다 호교ㅣ 소버먼트 티셔츠랑 같이입은 바지 제품명이랑사이즈좀 알수잇을까요?
노매뉴얼 가방 보자마자 바로 사러갑니다 ㅠㅠㅠㅠㅠ 가격대비 미쳤..
깡형 소버먼트 소개해 주셨을때 입으신 바지는 무엇인가요?
<a href="https://www.youtube.com/watch?v=vHdR1vnzhWI&amp;t=0m54s">0:54</a> 첫 코디에 매치하신 가방 정보 알 수 있을까요?
덕분에 가방하나 모자하나 줍줍해가용!
진짜 열심히사시는거 같아요 <br>항상 잘 보고있습니다.
잘보고 있습니다. 혹시 소버먼트 반팔 소개하실때 쓰신 팔찌 정보 알려주실수 있을까요?
깡형 좋은 제품 소개 감사합니다!! 혹시 첫번째 와이드 데님 어디껀지 알려줄 수 있나요?
주앙 옴므 옷 소개하실 때 입은 바지 어디 건지 알 수 있을까요
주앙.. 깡형 채널 진출이라니
깡형! 혹시 주앙옴므랑입은 바지정보좀 알수 있을까요 ㅠㅜ
노매뉴얼가방 계속 고민하다 할인해서 8만원일때도 참았는데 이번 깡님 할인에 쿠폰까지 해서 6만3천원이라 바로 샀네요! 항상 좋은 제품에 엄청난 할인 적용해주셔서 너무 감사합니다❤
깡형 슬리퍼 언제 출시예정인가요 ㄱㅐ굴!?
오늘도 옷 소개 감사합니다<br>항상 잘 보고 있어요<br>화이팅!!
볼캡 예쁜데 머리가 커서 안맞을까봐 걱정되네요.  대두 관련 모자 컨텐츠도 해주실 수 있나요
아 딱 제가 원하는 티가 보이네용. ㅎㅎ 바로 샀네용
어우 오늘도 열일하는 깡형 ~ 졸라리 멋있어요 형 ~~


127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /pred

형 레인부츠 추천 컨텐츠 부탁해도 돼? ㅜㅜ
소버먼트 소개할때 바지 어디껀지 궁금합니다 깡형
정말 너무 예쁜 옷 소개해주셔서 감사드립니다!!!
깡선생님 볼캡 진짜 갑이네요 감사합니당 ❤
너무 좋아요오옹~~~!❤️❤️❤️❤️❤️❤️❤️
주앙옴므 옷 정말 이쁘던데 세일까지 들어갔네 바로 사러 갈게욤 ㅎㅎ
깡형 항상 잘 보고 있습니다!! 제 패션에 생명을 더해주셔서 감사해용 :)
출챗
감사합니다 😊😊
옷추천은 역시 깡형..!!❤❤
1
옿
1등이닷!!!
1등 ㅎㅎ
1
추


127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 17:49:31] "POST /predict HTTP/1.1" 200 -


항상 좋은 옷 소개시켜주셔서 감사해요!
1등
1등❤
1빠
1등
Press Enter to stop the Flask app...
